In [7]:
import pandas as pd
import codecs
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

In [8]:
newspapers = pd.read_csv('C:/Users/jw156/Ironhack/vaccine/newspapers.csv', index_col=0)

In [247]:
newspapers.columns

Index(['hrefs', 'title', 'new_title', 'id', 'date', 'newspaper_name',
       'vaccine', 'pos_headline', 'lem_pos_headline'],
      dtype='object')

In [12]:
newspapers = newspapers.reset_index(drop=True)

# POS Determination of words

In [20]:
newspapers = newspapers.reset_index(drop=True)

In [68]:
import nltk
from nltk.stem import WordNetLemmatizer 

vaccine


In [22]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize


In [217]:
from nltk.corpus import wordnet as wn

def pos_headline(headline):
    new_headline = headline.split(' ')
    word_list = []
    for word in new_headline:
        if word == None or word.isalpha()==False:
            continue
        else:
            text = word_tokenize(word)
            postition_tag = nltk.pos_tag(text)
            word_list.append(postition_tag)
    return word_list

def is_proper_noun(tag):
    return tag in ['NNP', 'NNPS']

def is_noun(tag):
    return tag in ['NN', 'NNS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']



def penn_to_wn(headlines):
    headline_list = pos_headline(headlines)
    tagged_list = []
    for tag in headline_list:
        tag = list(tag[0])
        if is_adjective(tag[1]):
            tag[1] = (wn.ADJ)
            tagged_list.append(tag)
        elif is_noun(tag[1]):
            tag[1] =(wn.NOUN)
            tagged_list.append(tag)
        elif is_adverb(tag[1]):
            tag[1] =(wn.ADV)
            tagged_list.append(tag)
        elif is_verb(tag[1]):
            tag[1] =(wn.VERB)
            tagged_list.append(tag)
        elif is_proper_noun(tag[1]):
            tag[1] =('NNP')
            tagged_list.append(tag)
    return tagged_list

def lemming_words(headline):
    if len(headline) == 0:
        return 0
    headline_list = penn_to_wn(headline)    
    new_string = []
    lemmatizer = WordNetLemmatizer()
    for word in headline_list:
        if word[1] == 'NNP':
            continue
        else:
            word[0] = (lemmatizer.lemmatize(word[0], word[1]))
            new_string.append(word)

    return new_string

In [242]:
x = newspapers.title[1]
lemming_words(x)


[['Scepticism', 'n'],
 ['Oxford', 'n'],
 ['vaccine', 'n'],
 ['threatens', 'n'],
 ['immunisation', 'n'],
 ['push', 'n']]

In [86]:
newspapers['pos_headline'] = newspapers['title'].apply(pos_headline)

In [220]:
newspapers['lem_pos_headline'] = newspapers['title'].apply(lambda x: lemming_words(x))

In [243]:
newspapers['pos_headline'][1]

[[('Scepticism', 'NN')],
 [('over', 'IN')],
 [('Oxford', 'NN')],
 [('vaccine', 'NN')],
 [('threatens', 'NNS')],
 [('Europe', 'NNP'), ("'s", 'POS')],
 [('immunisation', 'NN')],
 [('push', 'NN')]]

# Word Sense Disambiguation

In [241]:
from nltk.wsd import lesk

x = 'The porn threatens to cause a lockdown'

print(lesk(x.split(), 'threatens', 'v'))


Synset('threaten.v.03')


In [282]:
def word_definitions(title, lemmed_title):
    synsets = []
    original_text = title
    lemmed_pos = lemmed_title
    for i in lemmed_pos:
        synsets.append(lesk(original_text.split(), i[0], i[1]))
    return synsets

In [1]:
x = newspapers['title'][1]
y = newspapers['lem_pos_headline'][1]
import re

j = word_definitions(x, y)
k = str(j[0])
print(k)
x = (re.findall(r"'([^']*)'", k))
f = swn.senti_synset(x[0])
# print(f)

NameError: name 'newspapers' is not defined

In [274]:
newspapers['word_definitions'] = newspapers.apply(lambda x: word_definitions(x['title'], x['lem_pos_headline']), axis=1)


In [276]:
newspapers

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[(Boris, NNP)], [(Johnson, NNP)], [(receives,...","[[receives, n], [Covid, n], [vaccine, n]]","[None, None, Synset('vaccine.n.01')]"
1,https://www.theguardian.com/world/2021/feb/19/...,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[(Scepticism, NN)], [(over, IN)], [(Oxford, N...","[[Scepticism, n], [Oxford, n], [vaccine, n], [...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
2,https://www.theguardian.com/world/2021/mar/18/...,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[(Thursday, NNP)], [(briefing, NN), (:, :)], ...","[[expert, n], [give, v], [Oxford, n], [vaccine...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
3,https://www.theguardian.com/world/2021/mar/16/...,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[(Oxford/AstraZeneca, NN)], [(vaccine, NN), (...","[[country, n], [have, v], [pause, v], [jab, n]]","[Synset('country.n.04'), Synset('take.v.35'), ..."
4,https://www.theguardian.com/world/2021/mar/16/...,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[(Chaos, NN)], [(in, IN)], [(Germany, NNP)], ...","[[Chaos, n], [suspension, n], [Oxford, n], [va...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...,...
62019,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62020,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62021,https://www.independent.co.uk/independentpremi...,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[(The, DT)], [(problem, NN)], [(with, IN)], [...","[[problem, n], [vaccine, n], [passport, n], [p...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."
62022,https://www.independent.co.uk/voices/editorial...,Reality is dawning on the EU: export bans are ...,real dawn eu export ban counterproduc cas covi...,uk,2021-03-25,independent,NaN,"[[(Reality, NN)], [(is, VBZ)], [(dawning, VBG)...","[[Reality, n], [be, v], [dawn, v], [export, n]...","[Synset('reality.n.03'), Synset('embody.v.02')..."
